## Import de bibliotecas necessárias para o funcionamento do script

In [11]:
import pandas as pd
import networkx as nx
import os

## Construção de arestas baseados nas igualdades de combinação e subgrupo e adicionando-as ao objeto com os nós

In [12]:
def gerar_edges(dados_para_conversao):
    edges = []
    for (index, item) in dados_para_conversao.iterrows():
        for index2, item2 in dados_para_conversao.iterrows():
            # 0 é o índice de código da escola
            if(item[0] != item2[0]):
                quantidade_de_associacoes = 0
                
                # 21 é o indice de combinações
                if(item[21] == item2[21]):
                    quantidade_de_associacoes = quantidade_de_associacoes+1

                # 9 é o indice de subgrupo
                if(item[9] == item2[9]):
                    quantidade_de_associacoes = quantidade_de_associacoes+1

                if(quantidade_de_associacoes > 0):
                    e = ((item[1]), (item2[1]), quantidade_de_associacoes)
                    edges.append(e)

    return edges

In [13]:
def gerar_atributos_de_rede(rede, arquivo_nome, urbano_ou_rural):
    
    global_efficiency = nx.global_efficiency(rede)
    local_efficiency = nx.local_efficiency(rede)
    modularity = nx.community.modularity(rede, nx.community.label_propagation_communities(rede))
    #fonte da densidade: networkx.org/documentation/stable/reference/generated/networkx.classes.function.density.html
    densidade = (2*len(rede.edges))/ (len(rede.nodes)*(len(rede.nodes)-1))
    assortativity = nx.degree_pearson_correlation_coefficient(rede)
    average_shortest_path = nx.average_shortest_path_length(rede)

    simrank_similarity = nx.simrank_similarity(rede)
    jaccard_coefficient = nx.jaccard_coefficient(rede)

    simrank_pd = pd.DataFrame(simrank_similarity)
    jaccard_coefficient_pd = pd.DataFrame(jaccard_coefficient)
    
    simrank_pd.melt()
    jaccard_coefficient_pd.melt()

    network_attributes = {
        'global_efficiency': global_efficiency,
        'local_efficiency': local_efficiency,
        'density': densidade,
        'modularity': modularity,
        'assortativity': assortativity,
        'average_shortest_path': average_shortest_path
    }

    network_attributes_df = pd.DataFrame(network_attributes.items())

    simrank_pd.to_csv("Resultados/"+arquivo_nome[:-5]+" - "+urbano_ou_rural+" - SimRank.csv")
    jaccard_coefficient_pd.to_csv("Resultados/"+arquivo_nome[:-5]+" - "+urbano_ou_rural+" - Jaccard.csv")
    network_attributes_df.to_csv("Resultados/"+arquivo_nome[:-5]+" - "+urbano_ou_rural+".csv")
    

## Importação dos dados contidos na mesma pasta e construção dos nós com seus respectivos valores 

In [14]:
def gerar_graph(arquivo_nome,urbano_ou_rural):
    dados_para_conversao = pd.read_excel(arquivo_nome, engine="openpyxl", sheet_name=urbano_ou_rural)

    G = nx.Graph()
    for (index, item) in dados_para_conversao.iterrows():
        G.add_node(item[1], 
               Nome_da_Escola=item[1],
               Nome_do_Municipio=item[2],
               Nome_UF=item[3],
               Tipo_de_dependencia_administrativa=item[4],
               Tipo_de_Localizacao=item[5],
               Quantidade_de_alunos=item[6],
               INSE_Valor_Absoluto=item[7],
               INSE_Classificação=item[8],
               Subgrupo=item[9],
               Proficiencia_em_Matematica=item[10],
               Nivel_de_proficiência_em_Matematica=item[11],
               Proficiencia_em_Lingua_Portuguesa=item[12],
               Nivel_de_proficiencia_em_Lingua_Portuguesa=item[13],
               IDEB=item[14],
               Indicador_de_Rendimento=item[15],
               Taxa_de_Aprovacao=item[16],
               Taxa_de_Reprovacao=item[17],
               Taxa_de_Abandono=item[18],
               Projecao=item[19],
               Atingiu_ou_nao_a_projeção=item[20],
               Perfil=item[21]
    )
    edges = gerar_edges(dados_para_conversao)    
    G.add_weighted_edges_from(edges)

    gerar_atributos_de_rede(G, arquivo_nome,urbano_ou_rural)
    nx.write_graphml(G, "Resultados/"+arquivo_nome+" - "+urbano_ou_rural+".graphml", encoding='utf-8', prettyprint=True)


# Importando todos os arquivos xlsx

In [15]:
path = os.getcwd()
files = os.listdir(path)
files_xls = [f for f in files if f[-4:] == 'xlsx']
for item in files_xls:
    # gerar_graph(item, "Urbana")
    gerar_graph(item, "Rural")
    

/Users/filipecruz/Library/Python/3.8/lib/python/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/Users/filipecruz/Library/Python/3.8/lib/python/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
